<a href="https://colab.research.google.com/github/a2c6201/machine-learning-book/blob/main/ch16/ch16_%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers
!wget https://storage.googleapis.com/what-if-tool-resources/lit-models/sst2_tiny.tar.gz
!tar -xvf sst2_tiny.tar.gz
!pip install lit_nlp tfds-nightly
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
--2023-07-05 06:31:57--  https://storage.googleapis.com/what-if-tool-resources/lit-models/sst2_tiny.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.107.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16362834 (16M) [application/octet-stream]
Saving to: ‘sst2_tiny.tar.gz’

sst2_tiny.tar.gz    100%[===================>]  15.60M  54.1MB/s    in 0.3s    

2023-07-05 06:31:57 (54.1 MB/s) - ‘sst2_tiny.tar.gz’ saved [16362834/16362834]

./
./tokenizer_config.json
./tf_model.h5
./config.json
./train.history.jso

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
from transformers import  AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup, AdamW, BertConfig
from tqdm.notebook import tqdm
from IPython.display import display, HTML
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

device = torch.device("cuda")

# データの準備

### IMDBデータセットを取得
すでにトークナイズ（文章をトークンに分解し、BERTに入力できる形にすること）されている？

In [22]:
from keras.datasets import imdb

SEED = 0

# データの読み込み
# label_dic : 0がnegativeで、1がpositive
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000, seed=SEED)  # データの読み込み

データの確認

In [28]:
# 単語を整数にマッピングする辞書
word_index = imdb.get_word_index()

# インデックスの最初の方は予約済み
word_index = {k:(v+3) for k,v in word_index.items()}
word_index[""] = 0
word_index[""] = 1
word_index[""] = 2  # unknown
word_index[""] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 [==============================] - 0s 0us/step


In [29]:
print('x_train.shape:', x_train.shape)
print('x_train[0]:', x_train[0])
print(decode_review(x_train[0]))
print('-----------------------')
print('y_train.shape:', y_train.shape)
print('-----------------------')
print('x_test.shape:', x_test.shape)
print('y_test:', y_test)

x_train.shape: (25000,)
x_train[0]: [1, 4078, 2, 9, 448, 725, 4, 2, 241, 2, 241, 38, 111, 2, 500, 40, 91, 374, 500, 679, 102, 13, 62, 135, 4, 2159, 92, 7241, 83, 6, 275, 3449, 66, 73, 5, 15, 271, 18, 14, 31, 99, 2149, 10, 10, 225, 6, 184, 196, 8127, 63, 2568, 5, 732, 4, 863, 18, 4, 65, 5, 4, 1397, 1111, 23, 63, 6, 564, 4892, 2, 5, 27, 476, 577, 2, 2, 2, 5, 492, 2, 2, 2, 216, 8, 847, 83, 4, 9370, 92, 168, 32, 99, 2575, 5, 515, 195, 481, 2017, 9, 348, 44, 4, 2, 23, 4, 1111, 8, 789, 2, 280, 4, 2, 517, 2, 10, 10, 1958, 5, 1364, 183, 380, 8, 140, 2, 5, 263, 6308, 140, 23, 6, 1973, 3318, 187, 4, 3450, 8, 1974, 618, 51, 9, 1448, 23, 14, 2, 1111, 19, 94, 680, 5818, 21, 11, 882, 25, 80, 24, 1414, 19, 803, 170, 23, 17, 2, 5, 2447, 2953, 79, 1376, 11, 8, 2, 4, 114, 60, 53, 51, 16, 66, 3742, 12, 83, 4, 5513, 7, 78, 212, 26, 94, 2, 1815, 2611, 46, 7, 4, 5797, 388, 63, 43, 2266, 5342, 2, 33, 94, 2099, 3002, 366, 45, 1852, 76, 303, 45, 31, 155, 269, 8, 216, 56, 5, 984, 142, 1393, 21, 160, 155, 399, 1

学習

In [ ]:

label_dic = {'ham': 0, 'spam': 1}  # spamを真値とする
label_dic_inv = {v: k for k, v in label_dic.items()}
label_ids = [label_dic[i] for i in labels]

# 7:3に学習データとテストデータを分割する(30%がテストデータ)
train_sentence, test_sentence, y_train, y_test = train_test_split(sentences, label_ids, test_size=0.3, random_state=SEED, stratify=label_ids)

## traintestspilit使う
## ham, spamを言い方悪い方に変える
## 文字数が多いのでエポックかかりそう